# Merge localisation from fluorophores that were on for multiple frames

**In STORM, PALM and similar localisation microscopy techniques, a fluorophore can be on for more than one frame and we would like to count them as one and get their average xy-position. ThunderSTORM, an imageJ plugin to analyse localisation microscopy data, has a function to merge such localisations and also provides multiple algorithms to do so. Although, in the process of merging, ThunderSTORM loses the identity of the particles that were merged and there is no simple way to check the merge quality. Here, I have a simple merge algorithm where I use nearest neighbour calculations in consecutive frames and assign a localisation within 60nm between two frames the same id (merge_id) while keeping their id intact. Later I use pandas groupby to group the merged localisations and explore their spatial distribution.**

## Load and clean data

In [1]:
# Here I import the python packages.
import pandas as pd
import numpy as np

As a first step in localisation microscopy the drift in the localisations is corrected. I have used ThunderSTORM to correct for the drift. Here I am loading the drift corrected localisation file.

In [2]:
# load drift corrected csv
df=pd.read_csv('df_ThunderSTORM_drift_corrected.csv')
df.head(2)

,id,frame,x [nm],y [nm],intensity [photon],bkgstd [photon]
0,1,1,8757.32179,17669.20288,744.71560,21.05830
1,2,1,9298.97079,15761.73288,45.41433,19.87785


In [3]:
# Here,I drop the columns that I do not need.
df.drop(columns=['intensity [photon]', 'bkgstd [photon]'], inplace = True)
df.head()

,id,frame,x [nm],y [nm]
0,1,1,8757.32179,17669.20288
1,2,1,9298.97079,15761.73288
2,3,1,11398.99679,17689.55988
3,4,1,11366.77179,22947.62888
4,5,1,11441.55479,12478.12388


In [4]:
# Here, I rename columns to have a simple name.
df.rename(columns={"x [nm]": "x", "y [nm]": "y"}, inplace=True)
print(df.shape)
df.head(2)

(180898, 4)


,id,frame,x,y
0,1,1,8757.32179,17669.20288
1,2,1,9298.97079,15761.73288


In [5]:
# I am only interested in the first 30000 frames so I slice the dataframe.
df=df[:30000]
df.shape

(30000, 4)

## Merge data

In [6]:
# Add an empty column named merge_id to the data frame
df["merge_id"] = np.nan
df["merge_distance"] = np.nan
df.head(2)

,id,frame,x,y,merge_id,merge_distance
0,1,1,8757.32179,17669.20288,NaN,NaN
1,2,1,9298.97079,15761.73288,NaN,NaN


In [7]:
# I would like to time my code so that I can improve the speed later on.
from timeit import default_timer as timer

In [8]:
# This is the main section of the code that does the merging.
# It may take a long time for large  files.
# Result of the loops influence the next loop and so may not be parallelised
# All calculations are vectorised to avoid 'for' loops and speed up things.
# only one for loop that goes over frames.

# scipy spatial is loaded so that I can calculate nearest neighbour distances
from scipy import spatial

start = timer() 

n=df['x'][df.frame == df.frame.unique()[0]].shape[0]
frame_number=df.frame.unique()

merge_id=np.arange(start=1, stop=n+1, step=1)
nearest_neighbor_distance_array=np.full(n, -1)

for i in range(len(frame_number)-1):
    # I first format/prepare the data for nearest neighbour calculations
    a = df['x'][df.frame == df.frame.unique()[i]].to_numpy()
    b = df['y'][df.frame == df.frame.unique()[i]].to_numpy()
    c = df['x'][df.frame == df.frame.unique()[i+1]].to_numpy()
    d = df['y'][df.frame == df.frame.unique()[i+1]].to_numpy()
    
    points = np.c_[a, b]    
    
    e = np.r_['0,2', c, d] # concatenate along first axis, dim>=2
    e = np.transpose(e)
    
    # calculate the neighbours, nearest neighbour distance and the neighbour id
    tree = spatial.KDTree(points)
    
    neighbors = tree.query_ball_point(e, 60) # neighbours within 60 nm 
    nearest_neighbors_distance = tree.query(e, distance_upper_bound=60)[0] # first nearest neighbour within 60 nm, distance
    nearest_neighbors_id= tree.query(e, distance_upper_bound=60)[1] # first nearest neighbour within 60 nm, id
    
    # merge id and merge distance is created
    for j in range(len(c)):
        if len(neighbors[j])==0:
            merge_id=np.append([merge_id], [max(merge_id)+1])
            # nearest neighbour distance is -1 when there is no neighbour in 60 nm
            nearest_neighbor_distance_array= np.append([nearest_neighbor_distance_array], -1)
        elif len(neighbors[j])>=1:
            neighbour_id=len(merge_id) - j - len(a) + nearest_neighbors_id[j]
            merge_id=np.append([merge_id], [merge_id[neighbour_id]])
            nearest_neighbor_distance_array= np.append([nearest_neighbor_distance_array], nearest_neighbors_distance[j])


print("time taken by this cell:", timer()-start) 

time taken by this cell: 26.6966125


In [9]:
# The merge id and neighbour distance is added to the dataframe as columns
df.merge_id = merge_id
df.merge_distance = nearest_neighbor_distance_array
df.tail() # I look at the bottom 5 rows of the dataframe

,id,frame,x,y,merge_id,merge_distance
29995,29996,12571,16517.69784,3249.75715,11710,-1.000000
29996,29997,12571,16602.29184,5785.32645,11678,39.046883
29997,29998,12571,16567.41084,10981.62375,6954,1.880448
29998,29999,12571,16519.55184,16267.28275,11697,28.736149
29999,30000,12571,19076.41684,10992.21375,11644,56.237877


In [10]:
df.to_csv('df.csv', index=False) # Dataframe df is saved with columns of merge_id and merge_distance

Pandas groupby function can group data according to their merge_id and make calculations for each group very efficiently.

In [11]:
# The rows are grouped by merge_id
grouped = df.groupby('merge_id')

In [12]:
# I create a merged dataframe where there is only 1 row for each fluorophore that was on for multiple consecutve frames
df_merged = grouped.agg({'x': 'mean', 'y': 'mean', 'merge_id': 'mean' } )
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11710 entries, 1 to 11710
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   x         11710 non-null  float64
 1   y         11710 non-null  float64
 2   merge_id  11710 non-null  int32  
dtypes: float64(2), int32(1)
memory usage: 320.2 KB


In [13]:
# Here I caculate the detections or the number of times the fluorophore was on in consecutive frames
df_merged["detections"]=grouped.size()

In [14]:
df_merged.head() # check the addition of detections column

,x,y,merge_id,detections
merge_id,,,,
1,8767.043607,17655.516087,1,3
2,9298.970790,15761.732880,2,1
3,11409.160656,17694.023673,3,2788
4,11366.771790,22947.628880,4,1
5,11441.554790,12478.123880,5,1


**For most purposes, this is the information you need. In the df_merged dataframe, now you have the mean x and y position of the fluorophores that were on for multiple frames. Then the detection column tells you how many frames the fluophores with each merge id were on for. The merge_id helps you link the df_merged and df dataframes and the identity of the original localisations involved in each merge_id can be traced back.**

### Merge quality check

I have corrected for drift but still a stationary fluorophore or a fiducial localisations may cover a small area and this area may represent the localisation error. Alternatively, a fluorphore might not be fixed in position as it may be tethered. In both cases the distribution of the localisations from each merge_id can give important information. Ideally a fiducial should have a symmetric localisation distribution while aconfined particle may or may not have a spherical distribution. Using groupby I have explored the symmetry and size of the localisation distribution. Alternatively, merge and groupby can be used to track particles and gets its diffusion properties.

In [15]:
from numpy import linalg as LA

X_std=[]
Y_std=[]
rhoRMSD=[]
lambda_max=[]
lambda_min=[]
symmetry_s=[]

start = timer() 

for name, group in grouped:
    
    X_std= np.append([X_std], [group['x'].std()])# standard deviation of the x locations
    Y_std= np.append([Y_std], [group['y'].std()])# standard deviation of the y locations 
    
    # I calculate the centre of mass and subtract it from localisations to getthe coordinates from the centre of mass
    x_centered=np.array(group['x']) - np.array(group['x'].mean())
    y_centered=np.array(group['y']) - np.array(group['y'].mean())
    
    # I use the centre of mass corrected coordinates to calculate the root mean square distance as a measure of size
    rhoRMSD_group=np.sqrt(np.mean(np.add(np.square(x_centered),np.square(np.array(y_centered)))))
    rhoRMSD= np.append([rhoRMSD], [rhoRMSD_group])
    
    # I use the centre of mass corrected coordinates to calculate the symmetry as a measure of shape/ellipticity
    group_count=np.array(group['x'].count())
    if (group_count > 2):
        eigen_values_lambda, _ = LA.eig(np.cov(x_centered, y_centered))
        lambda_max= np.append([lambda_max], [np.amax(eigen_values_lambda)]) 
        lambda_min=np.append([lambda_min], [np.amin(eigen_values_lambda)])
        symmetry_s=np.append([symmetry_s], [np.sqrt(np.amax(eigen_values_lambda)/np.amin(eigen_values_lambda))])
    else:
        lambda_max=np.append([lambda_max], -1)
        lambda_min=np.append([lambda_min], -1)
        symmetry_s=np.append([symmetry_s], -1)
        
print("time taken by this cell:", timer()-start)

# Here all the variables calculated are added to the dataframe as columns
df_merged["X_std"]=X_std
df_merged["Y_std"]=Y_std
df_merged["rhoRMSD"]=rhoRMSD
df_merged["lambda_max"]=lambda_max
df_merged["lambda_min"]=lambda_min
df_merged["symmetry_s"]=symmetry_s

df_merged.head(2) # visualise the dataframe

time taken by this cell: 6.869419199999996


,x,y,merge_id,detections,X_std,Y_std,rhoRMSD,lambda_max,lambda_min,symmetry_s
merge_id,,,,,,,,,,
1,8767.043607,17655.516087,1,3,10.81935,14.938601,15.060326,340.173237,0.046885,85.178956
2,9298.970790,15761.732880,2,1,NaN,NaN,0.000000,-1.000000,-1.000000,-1.000000


## Save dataframes df and df_merged

In [16]:
df.to_csv('df.csv', index=False) # Dataframe df is saved with a column of merge_id

In [17]:
df_merged.to_csv('df_merged.csv', index=False) # Dataframe df_merged is saved with a column of merge_id

**In this notebook, I have merged localisations from a fluorophore that was on for multiple frames. I have then used the merge_id to explore each particle positional distribution. The code can be used for particle tracking. The two tables/dataframes saved can be used to compute any summary statistic desired for each merge_id.**